In [1]:
# An investigation of Reed Solomon reliability for the bounded-fault scenario
# Tanj Bennett, copyright (c) 2023, Avant-Gray LLC
# Usage rights granted under terms of the MIT open-source license.

# using the very nice work of Tomer Fileba, whose work remains separate
# at https://github.com/tomerfiliba-org/reedsolomon/blob/master/src/reedsolo/reedsolo.py

# uncomment the following line once to install that library
# pip install --upgrade reedsolo
# pip install --upgrade numpy

In [2]:
# import and verify the import succeeded.

from reedsolo import RSCodec, ReedSolomonError
rsc = RSCodec(4)  # 4 ecc symbols.
rsc

In [3]:
# import and verify random number generation.  Nothing fancy is needed, just ensure it is present.

import random
import numpy

The encode operation will be used to take a 32-byte data set (held as a byte array) and then calculate 4 extra check symbols which are appended to the 36-byte stored form.  These 4 extra symbols are also bytes.  The Reed-Solomon (36,32) algorithm has the capability to use the 4 extra symbols to locate up to 2 errors in any of the 36 bytes and to calculate a correction for them.

If we know where the errors are then the 4 extra symbols can be used to correct up to 4 errors (these known locations are known as "erasures").  This does not help us for the DRAM case, since we rarely know where the errors are and it is unsafe to use that mode.  One of the sections of this code demonstrates use of erasure information.

The article "https://en.wikiversity.org/wiki/Reed%E2%80%93Solomon_codes_for_coders" is recommended as a very approachable and lucid explanation of Reed Solomon.  If you enjoyed math at the high school level you can read that article, just be prepared to be patient and methodical in working through it.

In [5]:
# Encoding
# just a list of numbers/symbols:
N = 32
x = bytearray(N)
for i in range(0,N) :
    x[i] = random.randint(0,255)

y = rsc.encode(x)

# used with N data bytes and 4 check symbols, this supports 256 bit data using RS(N+4,N)
len(x), x, len(y), y

(64,
 bytearray(b'\xb8.@\xd2\x8bO\xdc\xad\xdfE`\xf2\x92\xa8\xde\xdc\xf1G!\xac9\x13\x1d\xc9\xb1d\x1a\x97\xdf{\xf9o\x96P\x01\x1b1\t\x05tf\x9c\xa0>\xe6-\xc3\xa8\x85Wr\xfc2\\q\x89\xc2\xc8\x1c1T(vK'),
 68,
 bytearray(b'\xb8.@\xd2\x8bO\xdc\xad\xdfE`\xf2\x92\xa8\xde\xdc\xf1G!\xac9\x13\x1d\xc9\xb1d\x1a\x97\xdf{\xf9o\x96P\x01\x1b1\t\x05tf\x9c\xa0>\xe6-\xc3\xa8\x85Wr\xfc2\\q\x89\xc2\xc8\x1c1T(vK\x8d\x89\xbfh'))

The decode operation returns a 3-tuple, the [0] part of it is the corrected data.
The [1] part is the full data with check symbols, and the [2] part is the byte indexes of the corrections.
The decode operation will throw an exception if if detects a failure to deliver correct data.

In [ ]:
z = rsc.decode(y)
x, z, x == z[0]

 If we try to correct a bounded error (just 2 symbols aligned on a two-DQ boundary) then RS(N+4,N) should fix them all.  We count the attempts which raised no exception, and we verify the results claiming to deliver good data.

In [6]:
detections = 0
silentFaults = 0

# model 2-byte contiguous faults, so just half the positions
# the last bounded fault includes the check bytes
lastBound = int((N/2) + 1)

for trials in range (0,10000):
    # keep permuting the data to ensure we see impact from both the data and the errors
    for i in range(0,N) :
        x[i] = random.randint(0,255)

    y = rsc.encode(x)

    i = 2 * random.randint(0,lastBound)
    y[i] ^= random.randint(1,255)
    y[i+1] ^= random.randint(1,255)

    try:
        z = rsc.decode(y)
        if (z[0] != x):
            silentFaults += 1
    except:
        detections = detections + 1
        
trials, detections, silentFaults

(9999, 0, 0)

In [7]:
# The ratio of failures should be zero

shouldCorrectButFailed = (detections + silentFaults) / (trials + 1)
shouldCorrectButFailed

0.0

if we try to use RS(N+4,N) but overwhelm it with a whole-chip error (more than 2 bytes), how good is RS(N+4,N) at
reporting the uncorrectables?  We count the attempts which raised no exception, and verify that any that did not raise an exception are actually silent failures.

In [8]:
# scale up the error size to 4 contiguous bytes and check for detection of uncorrectables

detections = 0
silentFaults = 0

# the last bounded fault includes the check bytes
lastBound = int((N/4) + 0)

for trials in range (0,10000):
    # keep permuting the data to ensure we see impact from both the data and the errors
    for i in range(0,N) :
        x[i] = random.randint(0,255)

    y = rsc.encode(x)

    i = 4 *random.randint(0,lastBound)
    y[i] ^= random.randint(1,255)
    y[i+1] ^= random.randint(1,255)
    y[i+2] ^= random.randint(1,255)
    y[i+3] ^= random.randint(1,255)

    try:
        z = rsc.decode(y)
        if (z[0] != x):
            silentFaults += 1
    except:
        detections = detections + 1
        
trials, detections, silentFaults

(9999, 9645, 355)

In [ ]:
# The ratio of detection vs. deception attempts shows the quality of defense

probity = detections / (trials + 1)

# what is observed is around 99%: the RS(N+4,N) code will allow around 1% of whole-chip errors to pass silently undetected.

probity

How good is RS(N+4,N) at detecting separated uncorrectables?  These might be generated by RowHammer or similar disturbance effects, including electromagnetic interference.  We count the attempts which raised no exception and verify that they are silent failures.

This check runs over 256 bits, but modern CPUs and most other chips will use DD5 for 512 bit transfers.  This means that each half-transfer is a separate chance of detection.  If the system is secure it should be encrypting memory, which means distubance effects from other locations will affect bits in both halves, significantly raising the chances of detecting corruption.  Good RAS firmware should be alert to the repeated failures long before a successful corruptiong can be achieved.

In [9]:
detections = 0
silentFaults = 0
last = N + 3

for trials in range (0,10000):
    # keep permuting the data to ensure we see impact from both the data and the errors
    for i in range(0,N) :
        x[i] = random.randint(0,255)

    y = rsc.encode(x)

    i = random.randint(0,last)
    j = random.randint(0,last)
    while i == j:
        j = random.randint(0,last)
    k = random.randint(0,last)
    while (i == k) or (j == k):
        k = random.randint(0,last)
    m = random.randint(0,last)
    while (i == m) or (j == m) or (k == m):
        m = random.randint(0,last)
    n = random.randint(0,last)
    while (i == n) or (j == n) or (k == n) or (m == n):
        n = random.randint(0,last)

# comment out some of these to compare 3, 4, or 5 errors.  The ratio barely changes.
    y[i] ^= random.randint(1,255)
    y[j] ^= random.randint(1,255)
    y[k] ^= random.randint(1,255)
    y[m] ^= random.randint(1,255)
    y[n] ^= random.randint(1,255)

    try:
        z = rsc.decode(y)
        if (z[0] != x):
            silentFaults += 1
    except:
        detections = detections + 1
        
trials, detections, silentFaults

(9999, 9650, 350)

In [10]:
# The ratio of detection vs. deception attempts shows the quality of defense

probity = detections / (trials + 1)

# for RS(36,32) what is observed is about 99%: about 1% of multichip errors to pass silently.
# for RS(68,64) what is observed is about 99%: about 1% of multichip errors to pass silently.

probity

0.965

This last run models using R-S with erasures specifying each chip in turn, to see if it will reject wrong chips.
It is assuming we figured out which chip has failed and want to "run-flat" until a hardware repair is possible.
This would give hope of using the 9-chip position information to fix some full-chip errors with just 4 check symbols.
In general one could identify "chip" as a 4-byte bound, maybe a known-bad subarray.

It is not expected to work: it does not.  The decode function will never report a failure if it is given 4 locations to correct.  All 4 check symbols are consumed in calculating the erased locations, and none are left for verifying the work.  Despite intuition that multibit failures may be repeatable and thus the chip can be predicted (of tests after a fault can identify the fault) the actual field data is that most multibit failures are transient or intermittent.  Even if we put complex retry code into the RAS firmware of the host chip, that just raises complexity a lot but the reliability will not improve enough to change the memory into a higher grade of reliability.

Bounded fault ECC is quite possibly the right choice if savings on cost and power of memory are reinvested in other places for reliability, but it is going to give up on faults which an RS(40,32) system could fix.  It will, however detect >99% of the faults it cannot correct, making it quite trustworthy.  A 64GB DIMM with 36 DRAM chips would have a less than 1.5 FIT for silent faults, which is about 1 failure per 700 years.  On most systems that is much less than other causes of silent failures.

In [11]:
corrections = 0
silentFaults = 0
detections = 0
positions = 0
lastChip = int(N/4)
for trials in range (0,1000):
    # keep permuting the data to ensure we see impact from both the data and the errors
    x = y[0:N]
    y = rsc.encode(x)

    i = 4 * random.randint(0,lastChip)

    y[i]   ^= random.randint(1,255)
    y[i+1] ^= random.randint(1,255)
    y[i+2] ^= random.randint(1,255)
    y[i+3] ^= random.randint(1,255)

    for j in range (0,9):
        jj = j * 4
        positions += 1
        try:
            z = rsc.decode(y, erase_pos = [jj, jj+1, jj+2, jj+3])
            if(z[0] == x):
                corrections += 1
            else:
                silentFaults += 1
#                print(y)
        except:
            detections = detections + 1

trials, positions, corrections, silentFaults, detections


(999, 9000, 558, 8442, 0)

In [12]:
# we would need a ratio of 0.99 or better for this method to make sense

guidedBounds = corrections / (trials + 1)

# the R-S algorithm will hallucinate values if you specify the wrong erasure positions.
#    but it will be perfect if you have the correct erasures

guidedBounds

0.558

How good is RS(N+8,N) - commonly called "chipkill ECC" - at detecting multichip uncorrectables?  These might be generated by RowHammer or similar disturbance effects.

We count the attempts which raised no exception and verify any it claims to correct.  If you comment out the error-masking lines to leave only 4 faults you will observe (as expected) that RS(N+8,N) can correct four errors at any byte location, not just contiguous.

In [13]:
rsc8 = RSCodec(8)  # 8 ecc symbols, RS(N+8,N)

detections = 0
silentFaults = 0
last = N+7

for trials in range (0,10000):
    # keep permuting the data to ensure we see impact from both the data and the errors
    x = y[0:N]
    y = rsc8.encode(x)

    i = random.randint(0,last)
    j = random.randint(0,last)
    while i == j:
        j = random.randint(0,last)
    k = random.randint(0,last)
    while (i == k) or (j == k):
        k = random.randint(0,last)
    m = random.randint(0,last)
    while (i == m) or (j == m) or (k == m):
        m = random.randint(0,last)
    n = random.randint(0,last)
    while (i == n) or (j == n) or (k == n) or (m == n):
        n = random.randint(0,last)
    p = random.randint(0,last)
    while (i == p) or (j == p) or (k == p) or (m == p) or (n == p):
        p = random.randint(0,last)
    q = random.randint(0,last)
    while (i == q) or (j == q) or (k == q) or (m == q) or (n == q) or (p == q):
        q = random.randint(0,last)

# comment out some of these to compare 5, 6, or 7 errors.  They are all perfectly detected.
# all 4-location errors are corrected.

    y[i] ^= random.randint(1,255)
    y[j] ^= random.randint(1,255)
    y[k] ^= random.randint(1,255)
    y[m] ^= random.randint(1,255)
    y[n] ^= random.randint(1,255)
    y[p] ^= random.randint(1,255)
    y[q] ^= random.randint(1,255)

    try:
        z = rsc8.decode(y)
        if (z[0] != x):
            silentFaults += 1
    except:
        detections = detections + 1
        
trials, detections, silentFaults

(9999, 9999, 1)

In [14]:
# The ratio of detection vs. deception attempts shows the quality of defense

probity = detections / (trials + 1)

# what is observed is essentially perfect: the RS(40,32) code will detect all uncorrectable errors

probity

0.9999